In [8]:
!pip install flask-cors

In [9]:
import os
#import pandas as pd
#import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import wikipediaapi

from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy import distance

from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread


app = Flask(__name__)
CORS(app)

In [10]:
# import flask

In [11]:
custom_headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'accept-language': 'en-GB,en;q=0.9',
}

In [12]:
def get_request(url):
    res = requests.get(url, headers=custom_headers)
    return BeautifulSoup(res.text)

Wikipedia Amazon Warehouse Scraping

In [13]:
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'en')
page_py = wiki_wiki.page('List_of_Amazon_locations#North_America')

In [14]:
section = page_py.section_by_title('Fulfillment and warehousing')
us_housing = section.section_by_title('North America').section_by_title('United States')
full_housing_text = us_housing.text.split('\n')[3:]
# full_housing_text

In [15]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", 
               "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", 
               "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
               "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", 
               "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", 
               "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

amazon_warehouses = {}
sect = full_housing_text[0]
for loc in full_housing_text:
    # loc = loc.split()
    # print
    if loc in state_names:
        # print(loc)
        sect = loc
        amazon_warehouses[sect] = []
    else: 
        amazon_warehouses[sect] += [loc]

# amazon_warehouses

In [16]:
amazon_warehouses['Michigan']

['Brownstown Township (DTW5)',
 'Caledonia (GRR1)',
 'Delta Township',
 'Detroit',
 'Gaines Township',
 'Hazel Park (DDT1, SMI1) ',
 'Huron Township (DTW3) ',
 'Livonia (DET1)',
 'Plymouth (DTW8)',
 'Pontiac (DDT6, DET3)',
 'Romulus (DTW1, DTW3, DTW9, DDT2)',
 'Shelby Charter Township (DET2)',
 'Sterling Heights (DGR3)']

Finding Distances

In [17]:
def get_lat_lng_nominatim(city_name):
    url = f"https://nominatim.openstreetmap.org/search?q={city_name}&format=json"

    
    headers = {
        "User-Agent": "AmazonEco/1.0 (nmokh05@gmail.com)"
    }  # Replace with your actual app name and contact email
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if len(data) > 0:
                lat = data[0]['lat']
                lon = data[0]['lon']
                return float(lat), float(lon)
            else:
                print(f"No results found for city: {city_name}")
                return None, None
        else:
            print(f"Error {response.status_code}: {response.text}")
            return None, None
    except Exception as e:
        print("Error fetching geolocation: ", str(e))
        return None, None

In [18]:
origin_city = "Dearborn, MI"
origin_lat, origin_lng = get_lat_lng_nominatim(origin_city)
dearborn_mi = (origin_lat, origin_lng)

In [19]:
def get_distance_dearborn(city_name):
    dest_lat, dest_lng = get_lat_lng_nominatim(city_name)
    # dearborn_mi = (origin_lat, origin_lng)
    destination_coords = (dest_lat, dest_lng)
    return distance.distance(dearborn_mi, destination_coords).miles

Carbon Emission Calculator

In [20]:
def shipping_co2(distance, weight, transport):

    co2_api_key = "Onbqw4xVQ1bJnRqG9TlZuw"
    url = "https://www.carboninterface.com/api/v1/estimates"

    data = {
        "type": "shipping",
        "distance_unit": "mi",
        "distance_value": distance,
        "weight_unit": "g",
        "weight_value": weight,
        "transport_method": transport
    }

    headers = {
        "Authorization": f"Bearer {co2_api_key}",
        "Content-Type": "application/json"
    }

    try:

        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 201:
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print("Error calculating shipping emissions: ", str(e))
        return None

In [21]:
def results(response_data):
    
    if response_data and 'data' in response_data:
      #  estimate_id = response_data['data']['id']
        estimate_type = response_data['data']['type']
        attributes = response_data['data']['attributes']
        
        distance_value = attributes['distance_value']
        distance_unit = attributes['distance_unit']
        weight_value = attributes['weight_value']
        weight_unit = attributes['weight_unit']
        transport_method = attributes['transport_method']
        estimated_at = attributes['estimated_at']
        carbon_grams = attributes['carbon_g']
        carbon_pounds = attributes['carbon_lb']
        carbon_kilograms = attributes['carbon_kg']
        carbon_metric_tons = attributes['carbon_mt']
        
        # Displaying the extracted information
       # print(f"Estimate ID: {estimate_id}")
        # print(f"Type: {estimate_type}")
        # print(f"Distance: {distance_value} {distance_unit}")
        # print(f"Weight: {weight_value} {weight_unit}")
        # print(f"Transport Method: {transport_method}")
        # print(f"Estimated At: {estimated_at}")
        # print(f"Carbon Emissions: {carbon_grams} g, {carbon_kilograms} kg, {carbon_pounds} lb, {carbon_metric_tons} mt")
        return carbon_grams
    else:
        print("No data available.")

In [22]:
amazon_warehouses["Michigan"]

['Brownstown Township (DTW5)',
 'Caledonia (GRR1)',
 'Delta Township',
 'Detroit',
 'Gaines Township',
 'Hazel Park (DDT1, SMI1) ',
 'Huron Township (DTW3) ',
 'Livonia (DET1)',
 'Plymouth (DTW8)',
 'Pontiac (DDT6, DET3)',
 'Romulus (DTW1, DTW3, DTW9, DDT2)',
 'Shelby Charter Township (DET2)',
 'Sterling Heights (DGR3)']

In [23]:
def get_avg_dist_michigan():
    total_dist_michigan = 0
    for a_warehouse in amazon_warehouses["Michigan"]:
        full_loc = a_warehouse[2].split()[0] + ", MI"
        total_dist_michigan += get_distance_dearborn(full_loc)

    return total_dist_michigan / len(amazon_warehouses["Michigan"])

In [24]:
weight = 20000.00      # hard coding
transport = "truck"  # hard coding example of transportation
    
    # Calculate carbon emissions
response = shipping_co2(get_avg_dist_michigan, weight, transport)

# print(response)   
    # Display the results
our_results = results(response)

Error calculating shipping emissions:  Object of type function is not JSON serializable
No data available.


In [25]:
city = amazon_warehouses["Alabama"][2].split()[0] + ", Alabama"
my_distance = get_distance_dearborn(city)  # Example distance in miles
print(city)

weight = 20000.00      # hard coding
transport = "truck"  # hard coding example of transportation
    
    # Calculate carbon emissions
response = shipping_co2(my_distance, weight, transport)

# print(response)   
    # Display the results
our_results = results(response)

Madison, Alabama


In [26]:
def get_average_state(city, state):
    total_dist_state = 0
    for a_warehouse in amazon_warehouses[state]:
        full_loc = a_warehouse[2].split()[0] + ", " + state
        total_dist_state += get_distance_dearborn(full_loc)

    return total_dist_state / len(amazon_warehouses[state])

Putting it All Together

In [27]:
def get_location(url):
    full_soup = get_request(url)
    new_url = full_soup.find_all('div', class_ = "offer-display-feature-text")[2].find('span').find('a')

    if (new_url.text == "Amazon" or new_url.text == 'Amazon.com'):
        
        #get average from amazon warehouse data
        return get_avg_dist_michigan()
    
    half_pulled = re.findall(r'.+(\?ie.+)', new_url.get('href'))
    pulled_url = 'https://www.amazon.com/sp' + half_pulled[0]
    
    sec_soup = get_request(pulled_url)
    biz_address = sec_soup.find_all('div', attrs={'id': 'page-section-detail-seller-info'})[0].find_all('span')[-5:]
    list_biz_address = [span.text for span in biz_address]
    str_biz_address = " ".join(list_biz_address)

    return str_biz_address

In [36]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread

app = Flask(__name__)
CORS(app)  

@app.route('/calculate_co2', methods=['POST'])
def calculate_co2():
    data = request.get_json()  
    url = data.get('amazon_url')  

    city = get_location(url)  

    if isinstance(city, int):
        my_distance = city  
    else:
        city = city.split()[4:6]
        my_distance = get_distance_dearborn(city)  
        
    weight = 20000.00  
    transport = "truck"
    
    response = shipping_co2(my_distance, weight, transport)  
    our_results = results(response)  
    
    return jsonify({"co2": our_results})  

def run_flask():
    app.run(port=5000)

thread = Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
On macOS, try disabling the 'AirPlay Receiver' service from System Preferences -> General -> AirDrop & Handoff.


In [29]:
calculate_co2()

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [37]:
# my_distance = get_distance_dearborn("981 WOODLANE RD Jackson NJ 08527-1512 US")
# print("Dist: ", my_distance)

No results found for city: 981 WOODLANE RD Jackson NJ 08527-1512 US
Dist:  5871.479659782728


In [40]:
addr = "981 WOODLANE RD Jackson NJ 08527-1512 US"
addr = addr.split()[4:6]
my_distance = get_distance_dearborn(addr)
print("Dist: ", my_distance)

Dist:  460.22777633157335
